In [1]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
from tensorflow_hub.module_v2 import load

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')
print(tf.__version__)

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

c:\Users\Joshua\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Joshua\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.0 and is not supported. 
Some things might work, some things might not.
If you were to encount

2.9.0


In [2]:
train_ds = object_detector.DataLoader.from_pascal_voc(
    './project-6-at-2023-04-30-00-12-74270469/train',
    './project-6-at-2023-04-30-00-12-74270469/train',
    ['Counted_Objects', 'Not_Counted_Objects']
)

val_ds = object_detector.DataLoader.from_pascal_voc(
    './project-6-at-2023-04-30-00-12-74270469/validate',
    './project-6-at-2023-04-30-00-12-74270469/validate',
    ['Counted_Objects', 'Not_Counted_Objects']
)

In [3]:
print(train_ds.size)
print(val_ds.size)

108
27


In [4]:
#spec = model_spec.get('efficientdet_lite4')

spec = object_detector.EfficientDetSpec(
  model_name='efficientdet-lite4', 
  uri='https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/2', 
  hparams={'max_instances_per_image': 8000})

In [5]:
model = object_detector.create(train_ds, model_spec=spec, batch_size=8, train_whole_model=True, epochs=80, validation_data=val_ds)

Epoch 1/80
13/13 [==============================] - 287s 18s/step - det_loss: 1.7487 - cls_loss: 1.1201 - box_loss: 0.0126 - reg_l2_loss: 0.1078 - loss: 1.8565 - learning_rate: 0.0090 - gradient_norm: 0.5576 - val_det_loss: 1.6541 - val_cls_loss: 1.0861 - val_box_loss: 0.0114 - val_reg_l2_loss: 0.1078 - val_loss: 1.7619
Epoch 2/80
13/13 [==============================] - 229s 18s/step - det_loss: 1.6112 - cls_loss: 1.0411 - box_loss: 0.0114 - reg_l2_loss: 0.1078 - loss: 1.7190 - learning_rate: 0.0100 - gradient_norm: 0.8135 - val_det_loss: 1.3899 - val_cls_loss: 0.8924 - val_box_loss: 0.0099 - val_reg_l2_loss: 0.1078 - val_loss: 1.4976
Epoch 3/80
13/13 [==============================] - 232s 18s/step - det_loss: 1.2431 - cls_loss: 0.7722 - box_loss: 0.0094 - reg_l2_loss: 0.1078 - loss: 1.3509 - learning_rate: 0.0100 - gradient_norm: 1.9697 - val_det_loss: 1.3423 - val_cls_loss: 0.9062 - val_box_loss: 0.0087 - val_reg_l2_loss: 0.1078 - val_loss: 1.4501
Epoch 4/80
13/13 [================

In [6]:
model.evaluate(train_ds)

2/2 [==============================] - 59s 21s/step



{'AP': 0.5544611,
 'AP50': 0.76071715,
 'AP75': 0.6460187,
 'APs': 0.0,
 'APm': 0.0029702971,
 'APl': 0.5677828,
 'ARmax1': 0.17542134,
 'ARmax10': 0.4374202,
 'ARmax100': 0.6301941,
 'ARs': 0.0,
 'ARm': 0.0015789474,
 'ARl': 0.64533335,
 'AP_/Counted_Objects': 0.54408205,
 'AP_/Not_Counted_Objects': 0.5648402}

In [7]:
model.evaluate(val_ds)

1/1 [==============================] - 17s 17s/step



{'AP': 0.45551667,
 'AP50': 0.64833564,
 'AP75': 0.5302315,
 'APs': 0.0,
 'APm': 0.08316832,
 'APl': 0.45911354,
 'ARmax1': 0.13732842,
 'ARmax10': 0.3605559,
 'ARmax100': 0.5368201,
 'ARs': 0.0,
 'ARm': 0.10909091,
 'ARl': 0.5404842,
 'AP_/Counted_Objects': 0.44071186,
 'AP_/Not_Counted_Objects': 0.47032145}

In [8]:
model.export(export_dir='../obj_model_training/',tflite_filename='efd_4_uc_la_bs8.tflite')